# DH / DSPI

In [217]:
!pip install keras --upgrade
#!pip install tensorflow


Requirement already up-to-date: keras in /opt/conda/lib/python3.6/site-packages
Requirement already up-to-date: numpy>=1.9.1 in /opt/conda/lib/python3.6/site-packages (from keras)
    100% |████████████████████████████████| 2.8MB 230kB/s ta 0:00:011    43% |██████████████                  | 1.2MB 487kB/s eta 0:00:04    79% |█████████████████████████▌      | 2.2MB 3.6MB/s eta 0:00:01    85% |███████████████████████████▌    | 2.4MB 3.4MB/s eta 0:00:01
Requirement already up-to-date: keras-preprocessing>=1.0.5 in /opt/conda/lib/python3.6/site-packages (from keras)
    100% |████████████████████████████████| 31.2MB 22kB/s eta 0:00:011  3% |█                               | 1.1MB 3.3MB/s eta 0:00:10    7% |██▎                             | 2.3MB 3.7MB/s eta 0:00:08    31% |██████████                      | 9.7MB 3.1MB/s eta 0:00:07    32% |██████████▎                     | 10.0MB 3.2MB/s eta 0:00:07    34% |███████████▏                    | 10.9MB 4.1MB/s eta 0:00:05    41% |█████████████▍ 

In [409]:
import csv
import re
#from io import StringIO
import pandas as pd
import numpy as np

import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from sklearn.metrics import mean_absolute_error,matthews_corrcoef,classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from sklearn.utils import shuffle
from keras.models import Sequential
from keras import layers
#from keras.layers import Dense, Dropout, Activation
#from keras.layers.normalization import BatchNormalization
#from keras.layers.core import Activation
#from keras.layers.advanced_activations import PReLU
from keras.callbacks import CSVLogger,EarlyStopping, ModelCheckpoint
from keras import backend as K
import keras.losses




In [108]:
#Funciones
#Separa columna tomada de logs en campos
def splitdata(col):
    temp = []
    for i in col:
        t = (i.split('='))
        temp.append(t[0])           
    return temp

#Normmaliza Dataset:
def splitdf(df):
    df = df.ix[:, ~df.columns.str.contains('Unnamed')]
    #r = re.compile(r"[^a-zA-Z0-9-]")
    df2 = pd.DataFrame(columns=df.columns)
    for i in df:
        new = df[i].str.split('=',n=1, expand=True)
        df2[i] = new[1]#.str.extract(r'([a-zA-Z0-9=])')
        #df2[i] = r.sub("",new[1])
    return df2

def fortidf(df):
    lcol = [] 
    lcol = df.columns
    df.columns = splitdata(lcol)
    df2 = splitdf(df)
    df2['ioc'] = 0
    return df2

def iptoint(ip):
    tmp = re.findall(r"\d+", ip)
    iplist = "".join(map(str, tmp))
    return iplist

In [529]:
#Modelos
def nn_model():
    model = Sequential()
    model.add(Dense(100, init='uniform'))
    model.add(Activation('sigmoid'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(50, init='uniform'))
    model.add(Activation('sigmoid'))
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    model.add(Dense(1, init='uniform'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics=['accuracy', 'mse', f1_score, matthews_correlation])
    return(model)

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())


In [450]:
#Importa CSV a DF
#df = Logs identificados como maliciosos
#df2 = Logs de trafico
df2 = pd.read_csv('Traffic.csv')
df = pd.read_csv('botnet.csv')
df = fortidf(df)
df2 = fortidf(df2)


#Training
df3 = pd.read_csv('Training.csv')
df3 = fortidf(df3)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (44,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [451]:
#Columnas comunes entre logs de Trafico y logs de Botnet
scolumns=['date', 'time', 'devid', 'vd', 'type', 'subtype', 'action',
       'app', 'appcat','craction', 'crscore', 'devname',
       'dstcountry', 'dstintf', 'dstip', 'dstport', 'duration', 'dvid','level','logid', 'logver', 'policyid','proto', 'rcvdbyte',
       'rcvdpkt', 'sentbyte', 'sentpkt', 'service', 'sessionid', 'srccountry',
       'srcintf', 'srcip', 'srcport', 'threatcnts', 'threatlvls', 'threats',
       'threattyps', 'threatwgts', 'trandisp', 'tranip', 'tranport', 'ioc']

In [500]:
#Join de datos
#data = pd.concat([df,df2])
data = df3
data.sample(3)

,itime,date,time,devid,vd,type,subtype,action,app,appact,...,service,sessionid,srccountry,srcintf,srcip,srcport,trandisp,transip,transport,ioc
886,1545075955,2018-12-17,16:45:55,FG-5KD3915800565,REF1007243,traffic,forward,accept,DNS,detected,...,DNS,3488585671,Reserved,vl2197in,10.10.7.128,53460,snat,200.41.168.124,53460,0
3956,1545075761,2018-12-17,16:42:41,FG-5KD3915800565,REF1007243,traffic,forward,deny,1854/tcp,None,...,1854/tcp,3488081056,Switzerland,vl2196out,185.255.31.42,42474,dnat,None,None,0
11979,1545075113,2018-12-17,16:31:53,FG-5KD3915800565,REF1007243,traffic,forward,deny,Microsoft.Portal,drop-session,...,HTTPS,3485766089,Reserved,vl2197in,10.10.7.117,53076,snat,200.41.168.124,53076,0


In [501]:
#Seleccion de features
features = ['itime','srcip','dstip','dstport','duration', 'rcvdbyte',
       'rcvdpkt', 'sentbyte', 'sentpkt','service','ioc' ]
data.loc[:, features].sample(3)

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,service,ioc
24904,1545074024,10.10.8.218,200.0.243.10,53,60,353,1,65,1,DNS,0
4565,1545075703,10.10.8.103,200.0.243.10,53,60,275,1,69,1,DNS,0
31854,1545073464,10.10.4.45,200.0.243.11,53,60,202,1,62,1,DNS,0


In [508]:
#Clasifica infectados:
data.loc[data.appcat == 'Botnet', 'ioc'] = 1

#Elimina Nulls
data.loc[data.rcvdpkt.isnull(), 'rcvdpkt'] = 0
data.loc[data.sentpkt.isnull(), 'sentpkt'] = 0
data.loc[data.dstport.isnull(), 'dstport'] = 0
data.loc[data.service.isnull(), 'service'] = 'TCP'







In [509]:
data2 = data.loc[:, features]
data2.sample(3)

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,service,ioc
8442,1545075378,10.10.5.102,172.217.28.206,443,1,491,6,914,6,HTTPS,0
36017,1545073132,10.10.8.207,200.0.243.10,53,60,230,1,62,1,DNS,0
28582,1545073681,10.10.4.103,200.5.79.20,443,1,471,8,860,7,HTTPS,0


In [510]:
#Convierte IP en int
data2.srcip = data2.srcip.apply(iptoint)
data2.dstip = data2.dstip.apply(iptoint)


In [511]:
data2.isnull().any()

itime       False
srcip       False
dstip       False
dstport     False
duration    False
rcvdbyte    False
rcvdpkt     False
sentbyte    False
sentpkt     False
service     False
ioc         False
dtype: bool

In [512]:
#Convierte servicios en dummy
data2 = pd.get_dummies(data2, columns=['service'])
data2.sample(3)

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,ioc,...,service_SMTP,service_SMTPS,service_SNMP,service_SQUID,service_SSH,service_TCP,service_TCP_8888,service_TELNET,service_VNC,service_X-WINDOWS
12895,1545075037,10108102,2165822235,443,248,4356,9,979,7,0,...,0,0,0,0,0,0,0,0,0,0
35337,1545073176,10108218,17221728206,443,3,132,3,729,5,0,...,0,0,0,0,0,0,0,0,0,0
11831,1545075124,1010445,1923614422,123,60,76,1,76,1,0,...,0,0,0,0,0,0,0,0,0,0


In [543]:
y = data2['ioc'].values
X= data2.drop('ioc', axis=1)
X = X.values
#data2

In [514]:
data2

,itime,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,ioc,...,service_SMTP,service_SMTPS,service_SNMP,service_SQUID,service_SSH,service_TCP,service_TCP_8888,service_TELNET,service_VNC,service_X-WINDOWS
0,1545076015,1010867,1909815219,80,5,104,2,469,7,0,...,0,0,0,0,0,0,0,0,0,0
1,1545076015,10108207,200024310,53,60,506,1,70,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1545076015,10108207,200024310,53,60,528,1,63,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1545076015,10108207,200024310,53,60,370,1,66,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1545076015,10108207,200024310,53,60,376,1,76,1,0,...,0,0,0,0,0,0,0,0,0,0
5,1545076015,10108207,200024310,53,60,373,1,60,1,0,...,0,0,0,0,0,0,0,0,0,0
6,1545076015,10108207,200024310,53,60,230,1,62,1,0,...,0,0,0,0,0,0,0,0,0,0
7,1545076015,10107128,8919168180,443,61,312,6,52,1,0,...,0,0,0,0,0,0,0,0,0,0
8,1545076015,10108219,4010233226,443,2,1612,3,530,5,0,...,0,0,0,0,0,0,0,0,0,0
9,1545076015,10104102,2001096231,995,5,6543,11,818,8,0,...,0,0,0,0,0,0,0,0,0,0


In [535]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [544]:
#ntrain = data2.shape[0]
#X_train = X[:ntrain]
#X_test = X[ntrain:]
#Y_train = y[:ntrain]
#Y_test = y[ntrain:]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, stratify=y, test_size=0.4, random_state=123)


In [547]:
Y_test

array([0, 0, 0, ..., 0, 0, 0])

In [548]:
print (len(X_train),len(Y_train))
print (len(X_test),len(Y_test))

27231 27231
18155 18155


In [549]:
model = nn_model()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, kernel_initializer="uniform")`
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="uniform")`
  if sys.path[0] == '':


In [532]:
csv_logger = CSVLogger('log.txt')
earlyStopping = EarlyStopping(monitor='val_acc', patience=4, verbose=2, mode='min')

In [550]:
#model.fit(X_train.values,Y_train.values,nb_epoch=2,batch_size=128,callbacks=[csv_logger,earlyStopping],validation_data=(X_test.values,Y_test.values),verbose=1)
model.fit(X_train,Y_train,nb_epoch=2,batch_size=128,callbacks=[csv_logger,earlyStopping],validation_data=(X_test,Y_test),verbose=1)



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 27231 samples, validate on 18155 samples
Epoch 1/2
27231/27231 [==============================] - 6s 207us/step - loss: 0.0474 - acc: 0.9949 - mean_squared_error: 0.0192 - f1_score: nan - matthews_correlation: -3.7012e-05 - val_loss: 0.0435 - val_acc: 0.9932 - val_mean_squared_error: 0.0225 - val_f1_score: nan - val_matthews_correlation: 0.0000e+00
Epoch 2/2
27231/27231 [==============================] - 2s 82us/step - loss: 0.0438 - acc: 0.9943 - mean_squared_error: 0.0285 - f1_score: nan - matthews_correlation: 0.0000e+00 - val_loss: 0.0435 - val_acc: 0.9973 - val_mean_squared_error: 0.0261 - val_f1_score: nan - val_matthews_correlation: 0.0000e+00


In [551]:
model.fit(X_train,Y_train,nb_epoch=5,batch_size=500,validation_data=(X_test,Y_test),verbose=1)



Train on 27231 samples, validate on 18155 samples
Epoch 1/5
 5000/27231 [====>.........................] - ETA: 0s - loss: 0.0452 - acc: 0.9954 - mean_squared_error: 0.0298 - f1_score: nan - matthews_correlation: 0.0000e+00

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


27231/27231 [==============================] - 2s 57us/step - loss: 0.0439 - acc: 0.9947 - mean_squared_error: 0.0305 - f1_score: nan - matthews_correlation: 0.0000e+00 - val_loss: 0.0435 - val_acc: 0.9932 - val_mean_squared_error: 0.0274 - val_f1_score: nan - val_matthews_correlation: 0.0000e+00
Epoch 2/5
27231/27231 [==============================] - 1s 53us/step - loss: 0.0439 - acc: 0.9942 - mean_squared_error: 0.0309 - f1_score: nan - matthews_correlation: 0.0000e+00 - val_loss: 0.0435 - val_acc: 0.9932 - val_mean_squared_error: 0.0280 - val_f1_score: nan - val_matthews_correlation: 0.0000e+00
Epoch 3/5
27231/27231 [==============================] - 1s 54us/step - loss: 0.0439 - acc: 0.9943 - mean_squared_error: 0.0317 - f1_score: nan - matthews_correlation: 0.0000e+00 - val_loss: 0.0435 - val_acc: 0.9932 - val_mean_squared_error: 0.0289 - val_f1_score: nan - val_matthews_correlation: 0.0000e+00
Epoch 4/5
27231/27231 [==============================] - 1s 52us/step - loss: 0.0439 -

0.0027174910947082367